In [15]:
from bs4 import BeautifulSoup
import requests
import json
import os
import multiprocessing as mp

In [2]:
base_url="https://dblp.org/db/conf/acl/index.html"

In [ ]:
#Conference List Each Year
links_response = requests.get(base_url)
links=[]
if links_response.status_code == 200:
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(links_response.text, 'html.parser')
    for link in soup.select('ul > li > cite > a.toc-link'):
        links.append(link.get('href'))
    soup.decompose()


In [44]:
#Crawling Papers in Each Conference
paper_links=[]
for link in links:
    link_response = requests.get(link)
    if link_response.status_code == 200:
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(link_response.text, 'html.parser')
        temp_list=soup.select('ul > li.inproceedings > nav > ul > li > div > ul > li.ee > a')
        temp_list_len=len(temp_list)
        for i in range(0,temp_list_len):
            if temp_list[i].get('href').startswith('https://doi.org/'):
                paper_links.append(temp_list[i].get('href'))
        soup.decompose()
    elif link_response.status_code == 404:
        pass
    else:
        print("THERE IS AN ERROR!")
        break

## Use Semanticscholar API to get Paper informations
### https://github.com/danielnsilva/semanticscholar

In [ ]:
!pip install semanticscholar

In [ ]:
# paper doi만 추출
paper_DOI=[]
for paper_link in paper_links:
    paper_DOI.append(paper_link.split('https://doi.org/')[1])


In [212]:
#dblp_links, paper_links,paper_DOI
with open('./dblp_links.json','w') as f:
    json.dump(links,f)
with open('./paper_links.json','w') as f:
    json.dump(paper_links,f)
with open('./paper_doi.json','w') as f:
    json.dump(paper_DOI,f)

## API Only accepts 500 papers in one call.

## 약 10000개 paper 500개씩 Chunking & API CALL

In [74]:
# slice the list of paper doi
listOfPaper=[]
before=0
for i in range(500,len(paper_links),500):
    listOfPaper.append(paper_DOI[before:i])
    before=i
listOfPaper.append(paper_DOI[10000:len(paper_links)])


In [461]:
from semanticscholar import SemanticScholar
sch = SemanticScholar()

## 안에 있는 500개 중에 ERROR가 있어서 되는 chunk 먼저 API CALL

In [ ]:
## There is an error with API, so it needs to be modified a little bit
## 7~20 index first
temp7to9=[]
for i in range(7,10):
    print(i)
    paper=sch.get_papers(listOfPaper[i])
    temp7to9+=list(paper)
    

In [ ]:
## 10,11,12,13,14 is ERROR...............................ㅠㅠㅠㅠ
temp15to17=[]  ## typo!! 16까지임
for i in range(15,17):
    print(i)
    paper=sch.get_papers(listOfPaper[i])
    temp15to17+=list(paper)

In [107]:
#17,18,19 ERROR
temp20=[]
paper=sch.get_papers(listOfPaper[20])
temp20+=list(paper)

In [224]:
# listOfPapers
with open('listOfPaper.json','w') as f:
    json.dump(listOfPaper,f)

## SO many errors in API.....*(짧은 시간에 너무 많은 API call로 Ban당하는 중)*

In [435]:
import time # for time sleep
from tqdm import tqdm
paperIdList=[]# paperid 저장용 list size(15,500)

In [468]:
SEARCH_FIELDS = [
        'abstract',
        'authors',
        'citationCount',
        'corpusId',
        'externalIds',
        'fieldsOfStudy',
        'influentialCitationCount',
        'isOpenAccess',
        'journal',
        'openAccessPdf',
        'paperId',
        'publicationDate',
        'publicationTypes',
        'publicationVenue',
        'referenceCount',
        's2FieldsOfStudy',
        'title',
        'url',
        'venue',
        'year'
    ]

## 중간에 소문자로 인해 인식 안 되는 중 -> 변경

In [ ]:
list11to13=[] #11~13번 chunk
for i in [11,12,13]:
    ls=[]
    for j in tqdm(range(0,500),desc=str(i)):
        doi=listOfPaper[i][j].split("10.18653/v1/")[1].upper()
        ls.append("10.18653/v1/"+doi)
    list11to13.append(ls)    

In [ ]:
temp11to13=[]
for i in range(0,3):
    for j in tqdm(range(0,500),desc=str(i)):
        if j %10==0:
            time.sleep(20)
        try:
            paper=sch.get_paper(list11to13[i][j],SEARCH_FIELDS)
            temp11to13.append(paper)
        except:
            print(i,j,list11to13[i][j])

## 10.18653 이 아니라 10.3115인 구간으로 인해 (14,473)부터 다시 request

In [ ]:
list14=[] #14번 청크
for j in tqdm(range(0,473),desc=str(14)):
        doi=listOfPaper[14][j].split("10.18653/v1/")[1].upper()
        list14.append("10.18653/v1/"+doi)
for j in tqdm(range(473,500),desc=str(14)):
    doi=listOfPaper[14][j].split("10.3115/v1/")[1].upper()
    list14.append("10.3115/v1/"+doi)

In [477]:
temp14=[]
for j in tqdm(range(0,500),desc=str(i)):
        if j %10==0:
            time.sleep(20)
        try:
            paper=sch.get_paper(list14[j],SEARCH_FIELDS)
            temp14.append(paper)
        except:
            print(j,list14[j])

2:  26%|████████████████████▋                                                            | 128/500 [04:53<04:38,  1.33it/s]

127 10.18653/v1/P16-1146


2:  45%|████████████████████████████████████▎                                            | 224/500 [08:35<10:51,  2.36s/it]

223 10.18653/v1/P16-2011


2:  71%|█████████████████████████████████████████████████████████▏                       | 353/500 [13:24<07:59,  3.26s/it]

352 10.18653/v1/P16-3015


2: 100%|█████████████████████████████████████████████████████████████████████████████████| 500/500 [18:47<00:00,  2.25s/it]


## 17,18,19 chunk 작업

In [ ]:
list17to19=[]
for i in [17,18]:
    ls=[]
    for j in tqdm(range(0,500),desc=str(i)):
        doi=listOfPaper[i][j].split("10.3115/")[1].upper()
        ls.append("10.3115/"+doi)
    list17to19.append(ls) 
    
ls=[]
for j in tqdm(range(0,260),desc=str(19)):
    doi=listOfPaper[i][j].split("10.3115/")[1].upper()
    ls.append("10.3115/"+doi)
k=23
for i in range(260,282):
    ls.append("10.1007/3-540-55801-2_"+str(k))
    k+=1
for j in tqdm(range(282,500),desc=str(19)):
    doi=listOfPaper[19][j].split("10.3115/")[1].upper()
    ls.append("10.3115/"+doi)
list17to19.append(ls)

In [ ]:
temp17to19=[]
for i in range(0,3):
    for j in tqdm(range(0,500),desc=str(i)):
        if j %10==0:
            time.sleep(20)
        try:
            paper=sch.get_paper(list17to19[i][j],SEARCH_FIELDS)
            temp17to19.append(paper)
        except:
            print(i,j,list17to19[i][j])

0:  34%|███████████████████████████▍                                                     | 169/500 [06:19<03:09,  1.74it/s]

0 168 10.3115/1218955.1219038


0:  60%|████████████████████████████████████████████████▎                                | 298/500 [11:10<02:27,  1.37it/s]

0 297 10.3115/1073083.1073091


0:  65%|████████████████████████████████████████████████████▊                            | 326/500 [12:26<12:13,  4.21s/it]

...

2:  53%|██████████████████████████████████████████▉                                      | 265/500 [10:09<06:36,  1.69s/it]

2 263 10.1007/3-540-55801-2_26


2:  64%|███████████████████████████████████████████████████▋                             | 319/500 [12:07<04:53,  1.62s/it]

2 318 10.3115/981823.981860


2:  90%|████████████████████████████████████████████████████████████████████████▉        | 450/500 [17:02<00:24,  2.08it/s]

In [ ]:
for j in tqdm(range(385,500),desc=str(10)):
        doi=listOfPaper[10][j].split("10.18653/v1/")[1].upper() ## 대문자로 변경해야 됨
        listOfPaper[10][j]="10.18653/v1/"+doi

In [507]:
temp10=[]

for j in tqdm(range(0,500),desc=str(i)):
        if j %10==0:
            time.sleep(20)
        try:
            paper=sch.get_paper(listOfPaper[10][j],SEARCH_FIELDS)
            temp10.append(paper)
        except:
            print(i,j,listOfPaper[10][j])

4:   4%|██▌                                                                   | 18/500 [00:54<29:49,  3.71s/it]

4 17 10.18653/v1/2020.acl-main.519


4:  74%|██████████████████████████████████████████████████▉                  | 369/500 [14:03<01:15,  1.74it/s]

4 367 10.18653/v1/2020.acl-tutorials.6


4:  92%|███████████████████████████████████████████████████████████████▌     | 461/500 [17:46<04:09,  6.39s/it]

4 460 10.18653/v1/P19-1076


4: 100%|█████████████████████████████████████████████████████████████████████| 500/500 [18:55<00:00,  2.27s/it]


## 0~6 chunk는 중간에 계속 에러 나는 경우가 많아서 어쩔 수 없이 paperId필요
## 크롤링 사이트의 형식이 이상해서 가끔 제목추출이 이상하게 되는 경우가 있음. 
## 따라서 논문 제목을 위해 Crossref Unified Resource API 사용....
https://api.crossref.org/swagger-ui/index.html#/Works/get_works__doi_

In [ ]:
url="https://api.crossref.org/works/"+listOfPaper[0][0]
paper_response=requests.get(url)
paper=paper_response.json()


### DOI 중에 arkiv로 되어있는 DOI로 찾아야하는 경우가 많음. 따라서 semantic scholar에서 논문 제목을 통해 paperId 추출

In [531]:
paperIdList=[]
for i in [0,1,2,3,4,5,6]:
    ls=[]
    for j in tqdm(range(0,500),desc=str(i)):
            if j%10==0 and not (i==0 and j==0):
                time.sleep(20)
            url="https://api.crossref.org/works/"+listOfPaper[i][j]
            paper_response=requests.get(url)
            if paper_response.status_code==200:
                    paper=paper_response.json()
                    title=paper['message']['title'][0]
                    try:
                        temp=sch.search_paper(title)
                        if len(temp[0].paperId)==0:
                            ls.append(listOfPaper[i][j])
                            print(title,listOfPaper[i][j],i,j)
                        else:
                            ls.append(temp[0].paperId)
                            
                    except:
                        ls.append(listOfPaper[i][j])
                        print(title,listOfPaper[i][j],i,j)
            else:
                print("no")
    paperIdList.append(ls)
                        

0:  53%|████████████████████████████████████▌                                | 265/500 [33:14<20:42,  5.29s/it]

An Ordinal Latent Variable Model of Conflict Intensity 10.18653/v1/2023.acl-long.265 0 264


0:  53%|████████████████████████████████████▊                                | 267/500 [33:25<22:33,  5.81s/it]

Pre-Training to Learn in Context 10.18653/v1/2023.acl-long.267 0 266


0:  81%|███████████████████████████████████████████████████████▊             | 404/500 [46:59<21:03, 13.17s/it]

...

4:  92%|███████████████████████████████████████████████████████████████▍     | 460/500 [23:07<00:36,  1.09it/s]

Divide and Rule: Effective Pre-Training for Context-Aware Multi-Encoder Translation Models 10.18653/v1/2022.acl-long.312 4 459


5:  31%|█████████████████████▌                                               | 156/500 [07:39<26:38,  4.65s/it]

Sharpness-Aware Minimization Improves Language Model Generalization 10.18653/v1/2022.acl-long.508 5 155


6: 100%|█████████████████████████████████████████████████████████████████████| 500/500 [36:58<00:00,  4.44s/it]


In [ ]:
temp0to6=[]
for i in range(0,7):
    for j in tqdm(range(0,500),desc=str(i)):
        if j %10==0:
            time.sleep(20)
        try:
            paper=sch.get_paper(paperIdList[i][j],SEARCH_FIELDS)
            temp0to6.append(paper)
        except:
            print(i,j,paperIdList[i][j])

0:  39%|███████████████████████████▏                                         | 197/500 [08:06<20:48,  4.12s/it]

0 196 ade39c39048d66465c7288e4a7f8258a1bce9e60


0:  79%|██████████████████████████████████████████████████████▊              | 397/500 [16:00<02:17,  1.34s/it]

0 396 0232715f9089e3a2fc002cff6737bb9939805b8d


0:  93%|███████████████████████████████████████████████████████████████▉     | 463/500 [18:53<03:54,  6.34s/it]

0 462 52f3b181f6361cd85914798273e497264b2bc32d


2:  29%|████████████████████▎                                                | 147/500 [05:44<09:07,  1.55s/it]

2 146 ef5f7cd21b5d34797636239a7b9c8ba6af440aab


6:   4%|██▊                                                                   | 20/500 [00:45<04:07,  1.94it/s]

In [ ]:
for temp in [temp0to6,temp7to9,temp10,temp11to13,temp14,temp15to17,temp17to19,temp20]:
    print(len(temp))
    for i in range(0,len(temp)):
        temp[i]=dict(temp[i])

In [689]:
acl_papers=[]

acl_papers+=temp0to6

acl_papers+=temp7to9 #7~9 chunk

acl_papers+=temp10

acl_papers+=temp11to13

acl_papers+=temp14

acl_papers+=temp15to17 #15,16, '17' is typo!!!!

acl_papers+=temp17to19

acl_papers+=temp20 # 20 chunk

In [1120]:
datapath="./ACL_PAPERS.json"

In [ ]:
with open(datapath,'w') as f:
    json.dump(acl_papers,f)

----

## 300개 missing data 중 55개 제외 살림 (abstract data 없는게 있음)

In [1394]:
c=0
absls=[]
for i in range(0,len(acl_papers)):
        if acl_papers[i]['abstract']==None:
            print(i, 'none')
            absls.append(i)
            c+=1
        elif len(acl_papers[i]['abstract'])<2:
            print(i,'dot')
            absls.append(i)
            c+=1
print(c)

9684 none
9760 none
2


In [959]:
for i in absls:
    try:
        doi=acl_papers[i]['externalIds']['DOI'].split("10.18653/v1/")[1]
        link="https://aclanthology.org/"+doi
        link_response = requests.get(link)
    except:
        print(i,"no site")
        break
    if link_response.status_code == 200:
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(link_response.text, 'html.parser')
        abstract=soup.select('#main >div>div> div >div> span')
        try:
            if len(abstract[0].text)!=0:
                acl_papers[i]['abstract']=abstract[0].text
        except:
            print(i,"no abstract")
            break
        soup.decompose()
    elif link_response.status_code == 404:
        print(i)

4028 dd


# 약 79~99 - ACM library

In [1057]:
for i in absls:
    try:
        doi=acl_papers[i]['externalIds']['DOI']
        link="https://dl.acm.org/doi/abs/"+doi
        link_response = requests.get(link)
    except:
        print(i,"no site")
        break
    if link_response.status_code == 200:
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(link_response.text, 'html.parser')
        abstract=soup.select('div.abstractSection > p')
        try:
            if len(abstract[0].text)!=0:
                acl_papers[i]['abstract']=abstract[0].text
        except:
            print(i,"no abstract")
            break
        soup.decompose()
    elif link_response.status_code == 404:
        print(i)
    else:
        print("!!",i)

9760
9761
9762
9764
9765
9766
9767
9768
9769
9770
9771
9772
9773
9774
9775
9776
9777
9778
9779
9780
9781


# 9684,9760 제외

# 문제 발생

{2023: 1739,
 2022: 1266,
 2021: 1345,
 2020: 937,
 2019: 821,
 2018: 474,
 2017: 388,
 2016: 489,
 2015: 397,
 2014: 484,
 2013: 2,
 2006: 165,
 2005: 79,
 2004: 87,
 2002: 96,
 2003: 119,
 2001: 70,
 2000: 79,
 1999: 83,
 1998: 245,
 1997: 73,
 1996: 57,
 1995: 56,
 1994: 52,
 1993: 47,
 1992: 54,
 1991: 78,
 1990: 39,
 1989: 34,
 1988: 35,
 1987: 34,
 1986: 40,
 1985: 41,
 1984: 116,
 1983: 25,
 1982: 39,
 1981: 36,
 1980: 44,
 1979: 28}
 
 없는 년도가 있다...


## 당연히 DOI가 모든 paper에 있을 줄 알고 doi.org/~ 된 것만 추가함...

In [1127]:
#Crawling Papers in Each Conference
missed_links=[]
for link in tqdm(links):
    link_response = requests.get(link)
    if link_response.status_code == 200:
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(link_response.text, 'html.parser')
        temp_list=soup.select('ul > li.inproceedings > nav > ul > li > div > ul > li.ee > a')
        temp_list_len=len(temp_list)
        alreadyIn=0
        for i in range(0,temp_list_len):
            if temp_list[i].get('href').startswith('https://doi.org/'):
                alreadyIn=1
                break
        if alreadyIn==0:
            for i in range(0,temp_list_len):
                if temp_list[i].get('href').startswith('https://aclanthology.org/'):
                    missed_links.append(temp_list[i].get('href'))
        soup.decompose()
    else:
        print(link)
        

 80%|█████████████████████████████████████████████████████████▊              | 135/168 [05:52<01:26,  2.61s/it]

THERE IS AN ERROR!


In [ ]:
for link in tqdm(links[136:]):
    link_response = requests.get(link)
    if link_response.status_code == 200:
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(link_response.text, 'html.parser')
        temp_list=soup.select('ul > li.inproceedings > nav > ul > li > div > ul > li.ee > a')
        temp_list_len=len(temp_list)
        alreadyIn=0
        for i in range(0,temp_list_len):
            if temp_list[i].get('href').startswith('https://doi.org/'):
                alreadyIn=1
                break
        if alreadyIn==0:
            for i in range(0,temp_list_len):
                if temp_list[i].get('href').startswith('https://aclanthology.org/'):
                    missed_links.append(temp_list[i].get('href'))
        soup.decompose()
    else:
        print(link)

In [ ]:
link_response = requests.get("https://aclanthology.org/W97-1005/")
if link_response.status_code == 200:
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(link_response.text, 'html.parser')
        title=soup.select('#title > a')[0].text
        temp=soup.select('p.lead > a')
        authors=[{'name':i.text} for i in temp]
        year=soup.select('div.order-2 > dl > dd')[3].text


In [1191]:
#for abstraction
def paperId(title):
    temp=sch.search_paper(title)
    if temp[0]['title']==title:
        return temp[0]
    else :
        print(title)
        return None

In [1217]:
import random

In [ ]:
for link in tqdm(missed_links[2438:]):
    link_response = requests.get(link)
    if link_response.status_code == 200:
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(link_response.text, 'html.parser')
        title=soup.select('#title > a')[0].text
#         temp=soup.select('p.lead > a')
#         authors=[{'name':i.text} for i in temp]
#         year=soup.select('div.order-2 > dl > dd')[3].text
        paper_info=paperId(title)
        if paper_info is None:
            continue
        else:
            missed_papers.append(paper_info)
            x=random.randint(10, 15)
            time.sleep(x)
        soup.decompose()
    else:
        print(1,link)

In [1422]:
c=0
mls=[]
for i in range(0,len(missed_papers)):
        if missed_papers[i]['abstract']==None:
            print(i, 'none')
            mls.append(i)
            c+=1
        elif len(missed_papers[i]['abstract'])<2:
            print(i,'dot')
            mls.append(i)
            c+=1
print(c)

2177 none
2347 none
2391 none
3


In [1438]:
for i in missed_papers:
    try:
        if i['externalIds']['ACL']==None:
            print(i)
    except:
        print(i)

{'paperId': 'b8b961044536ca4e95b025d8683b6f3a096d256a', 'externalIds': {'DBLP': 'conf/acl/Alm11', 'MAG': '2251892406', 'CorpusId': 16940041}, 'corpusId': 16940041, 'publicationVenue': {'id': '1e33b3be-b2ab-46e9-96e8-d4eb4bad6e44', 'name': 'Annual Meeting of the Association for Computational Linguistics', 'type': 'conference', 'alternate_names': ['Annu Meet Assoc Comput Linguistics', 'Meeting of the Association for Computational Linguistics', 'ACL', 'Meet Assoc Comput Linguistics'], 'url': 'https://www.aclweb.org/anthology/venues/acl/'}, 'url': 'https://www.semanticscholar.org/paper/b8b961044536ca4e95b025d8683b6f3a096d256a', 'title': 'Subjective Natural Language Problems: Motivations, Applications, Characterizations, and Implications', 'abstract': 'This opinion paper discusses subjective natural language problems in terms of their motivations, applications, characterizations, and implications. It argues that such problems deserve increased attention because of their potential to challen

In [1430]:
acl_papers+=missed_papers